# QUACK Use Case 2 - Binary Clustering Demo (K=2)

## QAOA vs Simulated Annealing vs Gurobi

---

This notebook is a **demo** for **QUACK – Use Case 2**: binary clustering (K=2) on synthetic "polygon" instances.

It compares three solvers:

| Solver | Type | Description |
|--------|------|-------------|
| **Gurobi** | Exact (BQP) | Binary quadratic programming solver |
| **SA** | Classical heuristic | Simple Simulated Annealing |
| **QAOA** | Quantum heuristic | Quantum Approximate Optimization (CUDA-Q) |

### Instance format

Instances are stored as `.pkl` files with structure:
```python
{
    "distance_matrix": np.ndarray,  # (n, n) symmetric
    "y_true": np.ndarray,           # ground-truth labels
    "info": {"n_samples": int, ...}
}
```

### How to run this demo

1. Upload to Colab:
   - `CudaQ_clustering_k2_implementation.py`
   - This notebook
   - The `instances_k2/` folder with `.pkl` files

2. Set the `INSTANCES_DIR` path in **Section 3**

3. **Run all cells from top to bottom**

> **Quick start:** If you just want to run the full demo, upload your files, set the `INSTANCES_DIR` path in Section 3, and then run all cells from top to bottom.

---

## 2. Setup Environment (Colab)

This section installs the required packages.

**Notes:**
- Enable **GPU runtime** in Colab for best CUDA-Q performance
- Gurobi requires a license for large problems; if unavailable, SA + QAOA will still run
- Installation may take a few minutes

In [ ]:
# Install required packages for this demo.
# On Colab, it is recommended to enable a GPU runtime if you want to use CUDA-based backends.

!pip install -q cudaq scikit-learn dimod

# Try to make Gurobi available. If this fails (no license or other issue),
# the demo will still run with SA + QAOA only.
try:
    import gurobipy
    print("Gurobi already available.")
except ImportError:
    try:
        !pip install -q gurobipy
        print("Gurobi installed. Note: A license is required for large problems.")
    except Exception as e:
        print("Could not install Gurobi automatically.")
        print("The Gurobi-based solver will be disabled.")

In [ ]:
from pathlib import Path
import pickle
import time

import numpy as np
from sklearn.metrics import adjusted_rand_score

import cudaq

# Try importing Gurobi, otherwise mark it as unavailable
try:
    import gurobipy as gp
    from gurobipy import GRB
    GUROBI_AVAILABLE = True
except Exception:
    GUROBI_AVAILABLE = False
    print("[WARNING] Gurobi is not available. The Gurobi solver will be skipped.")

# Import QAOA helpers from the implementation module.
# Make sure 'CudaQ_clustering_k2_implementation.py' is in the same working directory.
from CudaQ_clustering_k2_implementation import (
    run_qaoa,
    pick_backend,
    shots_for_n,
    calculate_clustering_cost,
)

print("Imports completed.")

## 3. Set Paths and Load Instance

Set the path to your `instances_k2` folder.

**Options:**
- **Direct upload:** Upload `instances_k2` to `/content/instances_k2`
- **Google Drive:** Mount Drive and point to `"/content/drive/MyDrive/.../instances_k2"`

**⚠️ Edit `INSTANCES_DIR` below to match your setup!**

In [ ]:
# Set the path to the folder containing the .pkl instances.
# EDIT THIS PATH according to where you placed your 'instances_k2' folder in Colab.
#
# Example 1: you uploaded 'instances_k2' into the current working directory:
# INSTANCES_DIR = Path("/content/instances_k2")
#
# Example 2: you mounted Google Drive and stored instances there:
# INSTANCES_DIR = Path("/content/drive/MyDrive/QUACK/instances_k2")

INSTANCES_DIR = Path("/content/instances_k2")  # <-- EDIT THIS IF NEEDED


def list_available_instances():
    """Return a sorted list of available .pkl instances in INSTANCES_DIR."""
    if not INSTANCES_DIR.exists():
        raise FileNotFoundError(f"Instances directory not found: {INSTANCES_DIR}")
    instances = sorted(INSTANCES_DIR.glob("*.pkl"))
    if not instances:
        raise FileNotFoundError(f"No .pkl files found in {INSTANCES_DIR}")
    return instances


def load_instance(path: Path) -> dict:
    """Load a clustering instance from a pickle file.

    Expected structure:
        {
            "X": np.ndarray,                   # optional
            "y_true": np.ndarray,             # ground-truth labels (length n_samples)
            "distance_matrix": np.ndarray,    # (n_samples, n_samples)
            "distance_matrix_normalized": np.ndarray  # optional
            "info": {
                "n_samples": int,
                "n_clusters": int,
            }
        }
    """
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data


# Pick a default instance (first one in alphabetical order)
instances = list_available_instances()
instance_path = instances[0]
print(f"Using default instance: {instance_path.name}")

instance_data = load_instance(instance_path)

distance_matrix = instance_data["distance_matrix"]
y_true = instance_data["y_true"]
n_points = int(instance_data["info"]["n_samples"])

print(f"Loaded instance with n = {n_points} points.")

## 4. QAOA Solver (CUDA-Q)

This section defines and runs the **Quantum Approximate Optimization Algorithm (QAOA)** using CUDA-Q.

**Configurable parameters:**
- `backend_name`: `"nvidia"` (GPU) or `"qpp-cpu"` (CPU fallback)
- `p_layers`: Number of QAOA layers (circuit depth)
- `manual_shots`: Set to an integer to override automatic shot calculation, or `None` for auto
- `seed`: Random seed for reproducibility

In [ ]:
# QAOA configuration for the demo
backend_name = "nvidia"  # or "qpp-cpu" if GPU is not available
p_layers = 3
seed = 27
manual_shots = None  # set an integer to override the heuristic


def solve_qaoa(
    distance_matrix: np.ndarray,
    y_true: np.ndarray,
    n_points: int,
    backend_name: str = "nvidia",
    p_layers: int = 3,
    shots: int = None,
    seed: int = 27,
    iterations: int = 2000,
) -> dict:
    """
    Run QAOA using the existing 'run_qaoa' function from the implementation module.
    Returns a dictionary with solver name, best bitstring, cost, ARI, time, etc.
    """
    selected_backend = pick_backend(backend_name)
    print(f"[QAOA] Requested backend: {backend_name} -> Selected: {selected_backend}")

    if shots is None:
        shots = shots_for_n(n_points, base=3000)
    print(f"[QAOA] Using p = {p_layers}, shots = {shots}, seed = {seed}")

    start = time.perf_counter()

    best_cost, prob_best, ari, theta_used, results_dict, times, metrics = run_qaoa(
        distance_matrix=distance_matrix,
        y_true=y_true,
        n=n_points,
        p=p_layers,
        shots=shots,
        seed=seed,
        iterations=iterations,
        theta0=None,
        track_metrics=True,
        skip_vqe=False,
    )

    total_time = time.perf_counter() - start

    # Extract best bitstring (lowest cost)
    best_bitstring = min(results_dict, key=lambda b: results_dict[b][2])
    n_unique = len(results_dict)
    best_count = results_dict[best_bitstring][0]
    total_counts = sum(v[0] for v in results_dict.values())
    prob_empirical = best_count / total_counts if total_counts > 0 else 0.0

    print(f"[QAOA] Best cost: {best_cost:.4f}, ARI: {ari:.4f}, P(best): {prob_best:.3f}")

    return {
        "solver": "QAOA",
        "bitstring": best_bitstring,
        "cost": float(best_cost),
        "ari": float(ari),
        "time": float(total_time),
        "prob_best": float(prob_best),
        "prob_empirical": float(prob_empirical),
        "n_unique_states": int(n_unique),
        "theta": theta_used,
        "backend": selected_backend,
        "shots": shots,
        "times_breakdown": times,
    }

In [ ]:
qaoa_result = solve_qaoa(
    distance_matrix=distance_matrix,
    y_true=y_true,
    n_points=n_points,
    backend_name=backend_name,
    p_layers=p_layers,
    shots=manual_shots,
    seed=seed,
)
qaoa_result

## 5. Classical Solvers: Simulated Annealing and Gurobi

This section implements two classical baselines:

1. **Simulated Annealing (SA)** - A simple metaheuristic defined in this notebook
2. **Gurobi** - Exact solver (skipped if Gurobi is not available)

In [ ]:
def solve_sa(
    distance_matrix: np.ndarray,
    y_true: np.ndarray,
    n_points: int,
    n_steps: int = 5000,
    seed: int = 27,
) -> dict:
    """
    Very simple Simulated Annealing over bitstrings of length n_points.
    Uses 'calculate_clustering_cost' from the implementation module as energy function.
    This is a didactic baseline, not an optimized SA.
    """
    rng = np.random.default_rng(seed)

    # Random initial bitstring
    current = rng.integers(0, 2, size=n_points, dtype=int)
    current_cost = calculate_clustering_cost(distance_matrix, "".join(map(str, current)))

    best = current.copy()
    best_cost = current_cost

    T0 = 1.0
    alpha = 0.995
    T = T0

    start = time.perf_counter()

    for step in range(n_steps):
        idx = rng.integers(0, n_points)
        candidate = current.copy()
        candidate[idx] = 1 - candidate[idx]

        candidate_cost = calculate_clustering_cost(
            distance_matrix, "".join(map(str, candidate))
        )
        delta = candidate_cost - current_cost

        if delta < 0 or rng.random() < np.exp(-delta / max(T, 1e-8)):
            current = candidate
            current_cost = candidate_cost

        if current_cost < best_cost:
            best = current.copy()
            best_cost = current_cost

        T *= alpha

    t_sa = time.perf_counter() - start
    ari = adjusted_rand_score(y_true, best)

    print(f"[SA] Best cost: {best_cost:.4f}, ARI: {ari:.4f}, time: {t_sa:.2f}s")

    return {
        "solver": "SA",
        "bitstring": best,
        "cost": float(best_cost),
        "ari": float(ari),
        "time": float(t_sa),
    }


sa_result = solve_sa(distance_matrix, y_true, n_points, n_steps=5000, seed=seed)
sa_result

In [ ]:
def solve_gurobi(
    distance_matrix: np.ndarray,
    y_true: np.ndarray,
    n_points: int,
) -> dict:
    """
    Solve a simple binary quadratic clustering model with Gurobi.
    Uses a quadratic objective consistent with the distance-based cost.
    If Gurobi is not available, returns a placeholder result.
    """
    if not GUROBI_AVAILABLE:
        print("[Gurobi] Not available. Skipping Gurobi solver.")
        return {
            "solver": "Gurobi",
            "bitstring": None,
            "cost": float("nan"),
            "ari": float("nan"),
            "time": float("nan"),
        }

    model = gp.Model("binary_clustering_k2")
    model.Params.OutputFlag = 0  # silent

    x = model.addVars(n_points, vtype=GRB.BINARY, name="x")

    obj = gp.QuadExpr()
    for i in range(n_points):
        for j in range(i + 1, n_points):
            d_ij = float(distance_matrix[i, j])
            # Binary quadratic term similar to the cost used in the QAOA formulation
            obj += d_ij * (2 * x[i] * x[j] - x[i] - x[j] + 1)

    model.setObjective(obj, GRB.MINIMIZE)

    # Simple symmetry breaking: fix the first variable
    model.addConstr(x[0] == 0, name="symmetry_breaking")

    start = time.perf_counter()
    model.optimize()
    t_gurobi = time.perf_counter() - start

    best_bits = np.array([int(x[i].X + 0.5) for i in range(n_points)], dtype=int)
    cost = calculate_clustering_cost(distance_matrix, "".join(map(str, best_bits)))
    ari = adjusted_rand_score(y_true, best_bits)

    print(f"[Gurobi] Best cost: {cost:.4f}, ARI: {ari:.4f}, time: {t_gurobi:.2f}s")

    return {
        "solver": "Gurobi",
        "bitstring": best_bits,
        "cost": float(cost),
        "ari": float(ari),
        "time": float(t_gurobi),
    }


gurobi_result = solve_gurobi(distance_matrix, y_true, n_points)
gurobi_result

## 6. Run Full Demo and Compare Results

This section runs all three solvers in sequence and displays a comparison table.

**Metrics:**
- **Cost**: Total intra-cluster distance (lower is better)
- **ARI**: Adjusted Rand Index vs ground truth (1.0 = perfect match)
- **Time**: Wall-clock execution time

In [ ]:
import pandas as pd

print("========== QUACK Use Case 2 - K=2 Clustering DEMO ==========")
print(f"Instance: {instance_path.name}")
print(f"n_points: {n_points}")
print("============================================================\n")

# Run solvers
gurobi_result = solve_gurobi(distance_matrix, y_true, n_points)
sa_result = solve_sa(distance_matrix, y_true, n_points, n_steps=5000, seed=seed)
qaoa_result = solve_qaoa(
    distance_matrix,
    y_true,
    n_points,
    backend_name=backend_name,
    p_layers=p_layers,
    shots=manual_shots,
    seed=seed,
)

# Build summary table
summary_rows = []

if not np.isnan(gurobi_result["cost"]):
    summary_rows.append(
        {
            "solver": "Gurobi",
            "cost": gurobi_result["cost"],
            "ARI": gurobi_result["ari"],
            "time_s": gurobi_result["time"],
        }
    )

summary_rows.append(
    {
        "solver": "SA",
        "cost": sa_result["cost"],
        "ARI": sa_result["ari"],
        "time_s": sa_result["time"],
    }
)

summary_rows.append(
    {
        "solver": "QAOA",
        "cost": qaoa_result["cost"],
        "ARI": qaoa_result["ari"],
        "time_s": qaoa_result["time"],
    }
)

summary = pd.DataFrame(summary_rows)
display(summary)

print("\nLower cost is better. ARI close to 1.0 means closer to ground truth clustering.")
print("QAOA's 'prob_best' gives an estimate of how likely it is to sample the best-cost bitstring.\n")
print(f"QAOA backend: {qaoa_result['backend']}, shots: {qaoa_result['shots']}")
print(f"QAOA prob_best (model): {qaoa_result['prob_best']:.3f}, empirical: {qaoa_result['prob_empirical']:.3f}")
print(f"Unique measured states (QAOA): {qaoa_result['n_unique_states']}")

## Summary

This demo compared three approaches for binary (K=2) clustering:

| Solver | Type | Optimality |
|--------|------|------------|
| **Gurobi** | Exact (BQP) | Guaranteed optimal |
| **SA** | Classical heuristic | No guarantee |
| **QAOA** | Quantum heuristic | No guarantee |

**Key observations:**
- **Cost**: Lower is better. Gurobi provides the optimal reference.
- **ARI**: Adjusted Rand Index measures clustering agreement with ground truth.
- **P(best)**: Probability that QAOA samples the best solution - indicates circuit quality.

---

**Next steps:**
- Try different instances by modifying `instance_path = instances[INDEX]`
- Experiment with `p_layers` (more layers = deeper circuit, potentially better results)
- Compare GPU vs CPU backends

For the full benchmark suite, see `CudaQ_clustering_k2_implementation.py`.